In [50]:
import pandas as pd
import numpy as np
import re
import string as s
from datetime import datetime

PATH = 'C:\\Temp\\'
FILE_NAME_EXCEL = 'teste.xlsx'

def parse_datetime(x):
    dt = datetime.strptime(x, '%Y/%m/%d %H:%M:%S')
    return dt

def convToInt(val):
    return int(val)

def percentile(n):
    def _percentile(x):
        return np.percentile(x, n)
    _percentile.__name__ = '{}-th percentile'.format(n)
    return _percentile


In [64]:
jmeter_df = pd.read_csv('\\teste.jtl', 
                    delimiter=';', 
                    engine='python',
                    na_values='-', 
                   header=None,
                   names=['timeStamp','elapsed','label','responseCode','responseMessage','threadName','dataType',
                          'success','failureMessage','bytes','sentBytes',
                          'grpThreads','allThreads','Latency','IdleTime','Connect'],
                converters={'timeStamp': parse_datetime,
                           'elapsed': convToInt,
                           'bytes': convToInt,
                           'sentBytes': convToInt,
                           'grpThreads': convToInt,
                           'allThreads': convToInt,
                           'Latency': convToInt,
                           'IdleTime': convToInt,
                           'Connect': convToInt})
jmeter_df.set_index(['timeStamp'], inplace=True) 

In [73]:

gr_jmeter = jmeter_df.groupby(['timeStamp', 'label']).agg({
    
    'label': ['count'],
    'elapsed': ['max', 'mean',  percentile(90)]
})

teste = gr_jmeter.loc[gr_jmeter['elapsed'] >= 2]

teste.head()

NotImplementedError: Indexing a MultiIndex with a DataFrame key is not implemented

In [53]:
writer = pd.ExcelWriter(PATH + FILE_NAME_EXCEL, engine='xlsxwriter')
gr_jmeter.to_excel(writer, sheet_name='Plan1')
writer.save()